# Reaction rules extraction

After filtering the reaction data, the next step in SynPlanner training pipeline is the extraction of reaction rules. The reaction rules define the transformations that can be applied to synthesize target molecules.

### The extraction protocol
The protocol for extracting reaction rules from reactions in SynPlanner utilizes the CGRtools Python library. This procedure involves the following steps:

**1. CGR creation**

The input reaction is converted to the Condensed Graph of Reaction (CGR), which is a single graph encoding an ensemble of reactants and products. CGR results from the superposition of the atoms of products and reactants having the same numbers. It contains both conventional chemical bonds (single, double, triple, aromatic, etc.) and so-called “dynamic” bonds describing chemical transformations, i.e. breaking or forming a bond or changing bond order. Given CGRs it is possible to extract the reaction center of the reaction.

**2. Reaction center extraction**

Some reactions can have several reaction centers and if the multicenter_rules parameter is True, these centers will be included in a single reaction rule. Otherwise, each reaction center will be included in different reaction rules.

**3. Reaction center extension**

The extracted reaction center can be extended by the inclusion of neighboring atoms (environmental atoms) of radius N (can be specified by environment_atom_count parameter). Usually, the environment of radius 1 is included.

**4. Ring structures inclusion**

If the reaction center atoms are part of ring structures or functional groups, they can be included in the reaction center. Ring structures are identified by the Smallest Set of Smallest Rings (SSSR) algorithm implemented in CGRTools. If the include_rings parameter is True and any atom of the reaction center is part of the identified ring structure, the whole ring is included in the reaction center.

**5. Functional group inclusion**

Functional groups to be included in the reaction center should be specified manually as a list of their SMILES in func_groups_list in the configuration file. If include_func_groups is True, any atom of the reaction center is part of any specified functional group, and the whole functional group is included in the reaction center.

**6. Leaving/incoming groups inclusion**

Leaving and incoming groups can be included in the reaction center. These groups are identified by comparison of the atoms in reactants and products. If some atoms in reactants are not observed in products, these atoms are identified as a leaving group. Likewise, if some atoms in products are not observed in reactants, these atoms are identified as an incoming group. This functionality is supposed to handle protection/deprotection reactions and identify protective agents. Leaving groups are added to the reaction center if the keep_leaving_groups parameter is True and incoming groups are added to the reaction center if keep_incoming_groups parameter is True.

**7. Reagents inclusion**

Reaction rule can be further specified by the inclusion of reagents if the keep_reagents parameter is True. It means that structurally identical reaction rules with identical atom properties will be considered different if they are associated with different reagents.

**8. Atom properties specification**

Each atom in the extended and specified reaction center and its environment atoms (neighbor atoms) is described by four properties: the number of neighbors, hybridization type, the number of hydrogens, and the size of the ring (if the atom belongs to any ring). These properties determine the level of atom and reaction rule specification and can be disabled if needed in the atom_info_retention section in the configuration file. If some property in atom_info_retention is True, it means that this property of the atom will be included in the final reaction rule.

**9. Reaction rule creation**

After all settings, the final reaction rule is assembled, reversed (to be used in retrosynthesis mode), and validated by application of the final reaction rule to the original reaction, from which it was extracted.

**10. Reaction rule validation**

Finally, the extracted rules are filtered by popularity, which is defined by the min_popularity parameter. For example, min_popularity:3 means, that only rules observed in not less than 3 reactions from the reaction dataset are remained.

### Configuring reaction rule extraction
The `ExtractRuleConfig` class in SynPlanner allows for the fine-tuning of how reaction rules are extracted. Key parameters of this class include:

- **multicenter_rules**: Determines whether a single rule is extracted for all centers in multicenter reactions (`True`) or if separate rules are generated for each center (`False`).
- **include_rings**: Includes ring structures in the reaction rules connected to the reaction center atoms if set to `True`.
- **keep_leaving_groups**: Keeps the leaving groups in the extracted reaction rule when set to `True`.
- **keep_incoming_groups**: Retains incoming groups in the extracted reaction rule if set to `True`.
- **keep_reagents**: Includes reagents in the extracted reaction rule when `True`. Default is `False`.
- **environment_atom_count**: Sets the number of layers of atoms around the reaction center to be included in the rule. A value of `0` includes only the reaction center, `1` includes the first surrounding layer, and so on.
- **min_popularity**: Establishes the minimum number of times a rule must be applied to be included in further analysis.
- **atom_info_retention**: Dictates the level of detail retained about atoms in the reaction center and their environment. Default settings retain information about neighbors, hybridization, implicit hydrogens, and ring sizes for both the reaction center and its environment. Default settings:
    ```python
    
    {
        "reaction_center": {
            "neighbors": True,
            "hybridization": True,
            "implicit_hydrogens": False,
            "ring_sizes": False,
        },
        "environment": {
            "neighbors": False,
            "hybridization": False,
            "implicit_hydrogens": False,
            "ring_sizes": False,
        },
    }
    ```

These settings are crucial for tailoring the reaction rule extraction process to the specific needs of the retrosynthesis analysis, ensuring that the rules are both accurate and relevant.

## Introduction

The protocol for reaction rule extraction in SynPlanner includes several steps:

**1. CGR creation**

The input reaction is converted to the Condensed Graph of Reaction (CGR), which is a single graph encoding an ensemble of reactants and products. CGR results from the superposition of the atoms of products and reactants having the same numbers. It contains both conventional chemical bonds (single, double, triple, aromatic, etc.) and so-called “dynamic” bonds describing chemical transformations, i.e., breaking or forming a bond or changing bond order. Given CGRs it is possible to extract the reaction center of the reaction.

**2. Reaction center extraction**

Some reactions can have several reaction centers and if the `multicenter_rules` parameter is `True`, these centers will be included in a single reaction rule. Otherwise, each reaction center will be included in different reaction rules.

**3. Reaction center extension**

The extracted reaction center can be extended by the inclusion of neighboring atoms (environmental atoms) of radius N (can be specified by the `environment_atom_count` parameter). Usually, the environment of radius 1 is included.

**4. Ring structures inclusion**

If the reaction center atoms are part of ring structures or functional groups, they can be included in the reaction center. Ring structures are identified by the Smallest Set of Smallest Rings (SSSR) algorithm implemented in CGRTools. If the `include_rings` parameter is `True` and any atom of the reaction center is part of the identified ring structure, the whole ring is included in the reaction center.

**5. Functional group inclusion**

Functional groups to be included in the reaction center should be specified manually as a list of their SMILES in `func_groups_list` in the configuration file. If `include_func_groups` is `True`, any atom of the reaction center is part of any specified functional group, and the whole functional group is included in the reaction center.

**6. Leaving/incoming groups inclusion**

Leaving and incoming groups can be included in the reaction center. These groups are identified by comparison of the atoms in reactants and products. If some atoms in reactants are not observed in products, these atoms are identified as a leaving group. Likewise, if some atoms in products are not observed in reactants, these atoms are identified as an incoming group. This functionality is supposed to handle protection/deprotection reactions and identify protective agents. Leaving groups are added to the reaction center if the `keep_leaving_groups` parameter is `True` and incoming groups are added to the reaction center if `keep_incoming_groups` parameter is `True`.

**7. Reagents inclusion**

Reaction rules can be further specified by the inclusion of reagents if the `keep_reagents` parameter is `True`. It means that structurally identical reaction rules with identical atom properties will be considered different if they are associated with different reagents.

**8. Atom properties specification**

Each atom in the extended and specified reaction center and its environment atoms (neighbor atoms) is described by four properties: the number of neighbors, hybridization type, the number of hydrogens, and the size of the ring (if the atom belongs to any ring). These properties determine the level of atom and reaction rule specification and can be disabled if needed in the `atom_info_retention` section in the configuration file. If some property in `atom_info_retention` is `True`, it means that this property of the atom will be included in the final reaction rule.

**9. Reaction rule creation**

After all settings, the final reaction rule is assembled, reversed (to be used in retrosynthesis mode), and validated by application of the final reaction rule to the original reaction, from which it was extracted.

**10. Reaction rule validation**

Finally, the extracted rules are filtered by popularity, which is defined by the `min_popularity` parameter. For example, `min_popularity:3` means that only rules observed in not less than 3 reactions from the reaction dataset remain.

## Configuration

The reaction rules extraction protocol can be adjusted with the configuration yaml file:

```yaml
environment_atom_count: 1
min_popularity: 3
multicenter_rules: True
include_rings: True
keep_leaving_groups: True
keep_incoming_groups: False
keep_reagents: False
keep_metadata: False
atom_info_retention:
  reaction_center:
    neighbors: True
    hybridization: True
    implicit_hydrogens: False
    ring_sizes: False
  environment:
    neighbors: False
    hybridization: False
    implicit_hydrogens: False
    ring_sizes: False
```

**Configuration parameters**:

| Parameter                | Default | Description                                                                                                          |
|--------------------------|---------|----------------------------------------------------------------------------------------------------------------------|
| environment_atom_count   | 1       | Determines the number of layers of atoms around the reaction center to be included in the reaction rule. A value of 0 includes only the reaction center, 1 includes the first surrounding layer, and so on. |
| min_popularity           | 3       | Determines the minimum number of occurrences of a reaction rule in the reaction dataset.                              |
| multicenter_rules        | True    | Determines whether a single rule is extracted for all centers in multicenter reactions (True) or if separate rules are generated for each center (False). |
| include_rings            | True    | Includes ring structures in the reaction rules connected to the reaction center atoms if set to True.                 |
| keep_leaving_groups      | True    | Keeps the leaving groups in the extracted reaction rule when set to True.                                             |
| keep_incoming_groups     | False   | Retains incoming groups in the extracted reaction rule if set to True.                                                |
| keep_reagents            | False   | Includes reagents in the extracted reaction rule when True.                                                           |
| atom_info_retention      | --      | Dictates the level of detail retained about atoms in the reaction center and their environment. Default settings retain information about neighbors, hybridization, implicit hydrogens, and ring sizes for both the reaction center and its environment. |

## 1. Download input data

The input data will be downloaded from the [HuggingFace repository](https://huggingface.co/Laboratoire-De-Chemoinformatique/SynPlanner) to the current directory. 

In [ ]:
from pathlib import Path
from synplan.utils.loading import download_all_data

# replace this path where you want to keep all your results
root_folder = Path(".").resolve()  
root_folder.mkdir(root_folder, exist_ok=True)

download_all_data(save_to=root_folder)
tutorial_folder = root_folder.joinpath("tutorial")

## 2. Run extraction of rules

In [12]:
from synplan.utils.config import RuleExtractionConfig
from synplan.chem.reaction_rules.extraction import extract_rules_from_reactions

In [13]:
extraction_config = RuleExtractionConfig(
    min_popularity=3,
    environment_atom_count=1,
    multicenter_rules=True,
    include_rings=False,
    keep_leaving_groups=True,
    keep_incoming_groups=False,
    keep_reagents=False,
    atom_info_retention={
        "reaction_center": {
            "neighbors": True,  # retains information about neighboring atoms to the reaction center
            "hybridization": True,  # preserves the hybridization state of atoms at the reaction center
            "implicit_hydrogens": False,  # includes data on implicit hydrogen atoms attached to the reaction center
            "ring_sizes": False,  # keeps information about the sizes of rings that reaction center atoms are part of
        },
        "environment": {
            "neighbors": False,  # retains information about neighboring atoms to the atoms in the environment of the reaction center
            "hybridization": False,  # preserves the hybridization state of atoms in the environment
            "implicit_hydrogens": False,  # includes data on implicit hydrogen atoms attached to atoms in the environment
            "ring_sizes": False,  # keeps information about the sizes of rings that environment atoms are part of
        },
    },
)

This setup, for instance, retains leaving groups and specifies the level of detail for atom information retention around the reaction center. Each parameter in `ExtractRuleConfig` is designed to give users the flexibility to optimize rule extraction according to their requirements.

### Extracting reaction rules using `extract_rules_from_reactions`

After configuring the rule extraction settings in SynPlanner, the next step is to apply these configurations to extract reaction rules from the reaction data. This is achieved using the `extract_rules_from_reactions` function.

In [14]:
reaction_rules_path = root_folder.joinpath("reaction_rules.pickle")  # extracted reaction rules file

extract_rules_from_reactions(
    config=extraction_config,  # the configuration settings for rule extraction
    reaction_data_path=filtered_data_path,  # path to the reaction data file
    reaction_rules_path=reaction_rules_path,  # path to the pickle file where the extracted reaction rules will be stored
    num_cpus=4,
    batch_size=100,
)

Number of reactions processed: 69445 [04:10]


Number of extracted reaction rules: 21880


The extracted reaction rules can be loaded and visually inspected (the reaction rules are sorted by popularity):

In [15]:
with open(reaction_rules_path, "rb") as f:
    reaction_rules_list = [i[0] for i in pickle.load(f)]
for rule in reaction_rules_list:
    rule.clean2d()

In [16]:
reaction_rules_list[0]

In [17]:
reaction_rules_list[1]

In [18]:
reaction_rules_list[100]